In [29]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

import numpy as np
import time
from utils import model
from inference import log_marginal_likelihood
from inference import logmarglik
from inference import zstates as zs
from inference import zstates_old_method as zs_old
from utils import hyperparameters

nsample = 300
nsnps = 100
pi = 0.1
mu = 0.0
sigmabg = 0.001
sigma = 0.3
tau = 1 / (0.005 * 0.005)
prior = "gxpred-bslmm"

x, y, csnps, v = model.simulate(nsample, nsnps,
                                pi = pi,
                                mu = mu,
                                sigma = sigma,
                                sigmabg = sigmabg,
                                tau = tau)

nvar = x.shape[0]
nsample = x.shape[1]
params = np.array([pi, mu, sigma, sigmabg, tau])
scaledparams = hyperparameters.scale(params)
cmax = 1
#zstates_new = zs.create(scaledparams, x, y, cmax, nvar, 0.98)
zstates_old = zs_old.create(scaledparams, x, y, cmax, nvar, 0.98)
zstates_new = zstates_old

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
params = np.array([0.00495222, 0, 0.48975, 0.0680367, 1 / 0.00001 / 0.00001])
scaledparams = hyperparameters.scale(params)

start_time = time.time()
l, m = log_marginal_likelihood.full(scaledparams, x, y, zstates_old)
print ("Log marginal likelihood from full calculation: {:f}".format(m))
print("Calculated in {:f} seconds ---\n".format(time.time() - start_time))

start_time = time.time()
m = log_marginal_likelihood.func(scaledparams, x, y, zstates_old)
print ("Log marginal likelihood from fast calculation: {:f}".format(m))
print("Calculated in {:f} seconds ---\n".format(time.time() - start_time))

start_time = time.time()
m = log_marginal_likelihood.func(scaledparams, x, y, zstates_old)
print ("Log marginal likelihood using old method of zstates: {:f}".format(m))
print("Calculated in {:f} seconds ---\n".format(time.time() - start_time))

Log marginal likelihood from full calculation: -1355.547691
Calculated in 1.003454 seconds ---

Log marginal likelihood from fast calculation: -1355.547331
Calculated in 0.121942 seconds ---

Log marginal likelihood using old method of zstates: -1355.547331
Calculated in 0.124651 seconds ---



In [ ]:
zstates = zstates_new
#params = np.array([0.01, 0.1, 0.0003, 1.0, 1 / 0.5 / 0.5])
params = np.array([0.00495222, 0, 0.48975, 0.0680367, 1 / 0.0001 / 0.0001])
scaledparams = hyperparameters.scale(params)
m, der = log_marginal_likelihood.func_grad(scaledparams, x, y, zstates)

# Derivative of pi
delta = 0.00001
newparams = hyperparameters.scale(params)
newparams[0] += delta
newm = log_marginal_likelihood.func(newparams, x, y, zstates)
pi_grad = (newm - m) / delta
print ("Derivative of pi by brute force is {:f}".format(pi_grad))
print ("Derivative of pi from equation is {:f}".format(der[0]))

# Derivative of mu
delta = 0.001
newparams = hyperparameters.scale(params)
newparams[1] += delta
newm = log_marginal_likelihood.func(newparams, x, y, zstates)
mu_grad = (newm - m) / delta
print ("Derivative of mu by brute force is {:f}".format(mu_grad))
print ("Derivative of mu from equation is {:f}".format(der[1]))

# Derivative of sigma
delta = 0.0001
newparams = hyperparameters.scale(params)
newparams[2] += delta
newm = log_marginal_likelihood.func(newparams, x, y, zstates)
sigma_grad = (newm - m) / delta
print ("Derivative of sigma by brute force is {:f}".format(sigma_grad))
print ("Derivative of sigma from equation is {:f}".format(der[2]))

# Derivative of sigbg
delta = 0.0001
newparams = hyperparameters.scale(params)
newparams[3] += delta
newm = log_marginal_likelihood.func(newparams, x, y, zstates)
sigbg_grad = (newm - m) / delta
print ("Derivative of sigbg by brute force is {:f}".format(sigbg_grad))
print ("Derivative of sigbg from equation is {:f}".format(der[3]))

# Derivative of tau
delta = 0.0001
newparams = hyperparameters.scale(params)
newparams[4] += delta
newm = log_marginal_likelihood.func(newparams, x, y, zstates)
tau_grad = (newm - m) / delta
print ("Derivative of tau by brute force is {:f}".format(tau_grad))
print ("Derivative of tau from equation is {:f}".format(der[4]))

In [40]:
hyperpriors = [None, "L1", None, "L1", None]
hyperparams = {"lambda":0.01}

In [41]:
zstates = zstates_new
#params = np.array([0.01, 0.1, 0.0003, 1.0, 1 / 0.5 / 0.5])
params = np.array([0.00495222, 0, 0.48975, 0.6, 1 / 0.0001 / 0.0001])
scaledparams = hyperparameters.scale(params)
print(scaledparams)
success, m, der = logmarglik.func_grad(scaledparams, x, y, zstates, prior, hyperpriors, hyperparams)

# Derivative of pi
delta = 0.00001
newparams = hyperparameters.scale(params)
newparams[0] += delta
newm = logmarglik.func(newparams, x, y, zstates, prior, hyperpriors, hyperparams)
pi_grad = (newm - m) / delta
print ("Derivative of pi by brute force is {:f}".format(pi_grad))
print ("Derivative of pi from equation is {:f}".format(der[0]))

# Derivative of mu
delta = 0.1
newparams = hyperparameters.scale(params)
newparams[1] += delta
newm = logmarglik.func(newparams, x, y, zstates, prior, hyperpriors, hyperparams)
mu_grad = (newm - m) / delta
print ("Derivative of mu by brute force is {:f}".format(mu_grad))
print ("Derivative of mu from equation is {:f}".format(der[1]))

# Derivative of sigma
delta = 0.0001
newparams = hyperparameters.scale(params)
newparams[2] += delta
newm = logmarglik.func(newparams, x, y, zstates, prior, hyperpriors, hyperparams)
sigma_grad = (newm - m) / delta
print ("Derivative of sigma by brute force is {:f}".format(sigma_grad))
print ("Derivative of sigma from equation is {:f}".format(der[2]))

# Derivative of sigbg
delta = 0.0001
newparams = hyperparameters.scale(params)
newparams[3] += delta
newm = logmarglik.func(newparams, x, y, zstates, prior, hyperpriors, hyperparams)
sigbg_grad = (newm - m) / delta
print ("Derivative of sigbg by brute force is {:f}".format(sigbg_grad))
print ("Derivative of sigbg from equation is {:f}".format(der[3]))

# Derivative of tau
delta = 0.0001
newparams = hyperparameters.scale(params)
newparams[4] += delta
newm = logmarglik.func(newparams, x, y, zstates, prior, hyperpriors, hyperparams)
tau_grad = (newm - m) / delta
print ("Derivative of tau by brute force is {:f}".format(tau_grad))
print ("Derivative of tau from equation is {:f}".format(der[4]))

[-5.3029548   0.         -0.71386022 -0.51082562 -9.21034037]
Derivative of pi by brute force is 0.220537
Derivative of pi from equation is 0.220503
Derivative of mu by brute force is 0.009943
Derivative of mu from equation is -0.000060
Derivative of sigma by brute force is 0.598729
Derivative of sigma from equation is 0.599063
Derivative of sigbg by brute force is 97.304285
Derivative of sigbg from equation is 97.345633
Derivative of tau by brute force is -199979814.267457
Derivative of tau from equation is -199999813.421095


In [ ]:
der

In [ ]:
mu_grad